Current and near-term quantum computers suffer from imperfections, as we repeatedly pointed it out. This is why we cannot run long algorithms, that is, deep circuits on them. A new breed of algorithms started to appear since 2013 that focus on getting an advantage from imperfect quantum computers. The basic idea is extremely simple: run a short sequence of gates where some gates are parametrized. Then read out the result, make adjustments to the parameters on a classical computer, and repeat the calculation with the new parameters on the quantum hardware. This way we create an iterative loop between the quantum and the classical processing units, creating classical-quantum hybrid algorithms.

<img src="../figures/hybrid_classical_quantum.svg" alt="Hybrid classical-quantum paradigm" style="width: 400px;"/>

These algorithms are also called variational to reflect the variational approach to changing the parameters. One of the most important examples of this approach is the quantum approximate optimization algorithm, which is the subject of this notebook.

# Quantum approximate optimization algorithm

The quantum approximate optimization algorithm (QAOA) is a shallow-circuit variational algorithm for gate-model quantum computers that was inspired by quantum annealing. We discretize the adiabatic pathway in some $p$ steps, where $p$ influences precision. Each discrete time step $i$ has two parameters, $\beta_i, \gamma_i$. The classical variational algorithms does an optimization over these parameters based on the observed energy at the end of a run on the quantum hardware.

More formally, we want to discretize the time-dependent $H(t)=(1-t)H_0 + tH_1$ under adiabatic conditions. We achieve this by Trotterizing the unitary. For instance, for time step $t_0$, we can split this unitary as $U(t_0) = U(H_0, \beta_0)U(H_1, \gamma_0)$. We can continue doing this for subsequent time steps, eventually splitting up the evolution to $p$ such chunks:

$$
U = U(H_0, \beta_0)U(H_1, \gamma_0)\ldots U(H_0, \beta_p)U(H_1, \gamma_p).
$$

At the end of optimizing the parameters, this discretized evolution will approximate the adiabatic pathway:

<img src="../figures/qaoa_process.svg" alt="Quantum approximate optimization algorithm" style="width: 400px;"/>

The Hamiltonian $H_0$ is often referred to as the driving or mixing Hamiltonian, and $H_1$ as the cost Hamiltonian. The simplest mixing Hamiltonian is $H_0 = -\sum_i \sigma^X_i$, the same as the initial Hamiltonian in quantum annealing. By alternating between the two Hamiltonians, the mixing Hamiltonian drives the state towards an equal superposition, whereas the cost Hamiltonian tries to seek its own ground state.

Let us import the necessary packages first:

In [ ]:
import itertools
import numpy as np
from qiskit import Aer, execute
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import *
from scipy.optimize import minimize
np.set_printoptions(precision=3, suppress=True)

Now we can define our mixing Hamiltonian on some qubits. As in the notebook on classical and quantum many-body physics, we had to define, for instance, an `IZ` operator to express $\mathbb{I}\otimes\sigma_1^Z$, that is, the $\sigma_1^Z$ operator acting only on qubit 1. We can achieve the same effect the following way (this time using the Pauli-X operator):

In Qiskit, Pauli matrices can be instantiated using the class `Pauli`. This class takes two parameters, the first for $\sigma^Z$ and the second for $\sigma^X$. Each parameter is a binary vector of dimension `n_qubits`, such that the component $i$ is 1 if you want a Pauli matrix to apply on the $i^{th}$ qubit and 0 otherwise. For instance, $\sigma_1^Z \otimes \sigma_3^Z \otimes \sigma_1^X$ would be implemented using `Pauli([1,0,1], [1,0,0])`.

In order to build Hamiltonians from Pauli matrices and make them evolve (i.e. exponentiate them, as required in QAOA), we will use the operator flow functionality in Qiskit Aqua. The class `PauliOp` can be used to construct Hamiltonians from Pauli matrices. It takes a `Pauli` object and a coefficient and represents the operator `coefficient * Pauli`. Operators represented by `PauliOp` objects can be simply added together with `+` to represent weighted sums of Pauli matrices. For instance, $3 \sigma^Z_1 + 2 \sigma^X_3$ would be written `PauliOp(Pauli([1,0,0], [0,0,0]), 3) + PauliOp(Pauli([0,0,0], [0,0,1]), 2)`. The resulting object of this summation is an object of `SummedOp` type: the class `SummedOp` in Qiskit Aqua is used to represent sums of various operators. We can as well use the `SummedOp` class directly to construct the weighted sum of Pauli matrices instead of doing it via the `+` sign: `SummedOp([PauliOp(Pauli([1,0,0], [0,0,0]), 3), PauliOp(Pauli([0,0,0], [0,0,1]), 2)])`.

To simplify the code, let's build a function `pauli_x` that simply takes a qubit and a coefficient and returns the corresponding Pauli-X operator as a `PauliOp`  

In [ ]:
def pauli_x(qubit_index, coeff):
    eye = np.eye((n_qubits)) # the i^th row of the identity matrix is the correct parameter for \sigma_i 
    return PauliOp(Pauli(np.zeros(n_qubits), eye[qubit_index]), coeff)

The coefficient here means the strength of the transverse field at the given qubit. This operator will act trivially on all qubits, except the given one. Let's define the mixing Hamiltonian over two qubits:

In [ ]:
n_qubits = 2
Hm = SummedOp([pauli_x(i, -1) for i in range(n_qubits)])

As an example, we will minimize the Ising problem defined by the cost Hamiltonian $H_c=-\sigma^Z_1 \otimes \sigma^Z_2$. First let's create the functions defining the operators using the Pauli-Z matrix:

In [ ]:
def pauli_z(qubit_index, coeff):
    eye = np.eye((n_qubits))
    return PauliOp(Pauli(eye[qubit_index], np.zeros(n_qubits)), coeff)


def product_pauli_z(q1, q2, coeff):
    eye = np.eye((n_qubits))
    return PauliOp(Pauli(eye[q1], np.zeros(n_qubits)) * Pauli(eye[q2], np.zeros(n_qubits)), coeff)

Then we define the cost Hamiltonian:

In [ ]:
J = np.array([[0,1],[0,0]])

Hc = SummedOp([product_pauli_z(i, j, -J[i,j]) 
                  for i,j in itertools.product(range(n_qubits), repeat=2)])

We set the number of time evolution steps $p=1$ and initialize the $\beta_i$ and $\gamma_i$ parameters:

In [ ]:
p = 1
beta = np.random.uniform(0, np.pi*2, p)
gamma = np.random.uniform(0, np.pi*2, p)

The initial state is a uniform superposition of all the states $|q_1,...,q_n\rangle$

To declare the initial state, we use the Qiskit Aqua class `Custom`. It takes two arguments: the number of qubits of the state we want to prepare, and the vector containing the amplitudes.

In [ ]:
init_state_vect = [1 for i in range(2 ** n_qubits)]
init_state = VectorStateFn(init_state_vect, coeff=1/np.sqrt(2 ** n_qubits))

Now, we define a function `evolve` that takes a Hamiltonian $H$ and an angle $t$ and returns an operator representing the unitary matrix $e^{-i H t}$. 

For that, we use the method `exp_i()` of the operator classes in Qiskit Aqua. Note that this function does not accept any arguments, particularly it does not accept evolution angle, it just exponentiates `-i` times whatever it is called on, so before the call to this function we multiply the operator with the necessary angle

In [ ]:
def evolve(hamiltonian, angle):
    return (angle * hamiltonian).exp_i()

To create the entire sequence of operators for our QAOA circuit, we need to compose the different unitary matrices given by `evolve`, then compose the result with the initial state. In Qiskit Aqua operator composition basically constructs the sequence in which the operators need to act on the initial state. For instance, if we want to act the operator `A` on `init_state` then act the operator `B`, then this can be writter as `B @ A @ init_state`, or by using the `ComposedOp` facility directly `ComposedOp([B, A, init_state])`. 

In [ ]:
def create_circuit(beta, gamma):
    return ComposedOp([evolve(Hm, beta[i]) @ evolve(Hc, gamma[i]) for i in range(p)] + [init_state])

We now create a function `evaluate_hamiltonian` that takes a single vector `gamma_beta` (the concatenation of `gamma` and `beta`) and a `hamiltonian` and returns $\langle H_c \rangle = \langle \psi | H_c | \psi \rangle$ where $\psi$ is defined by the circuit created with the function above.

We use the class `OperatorStateFn` to construct the measurement operator for the corresponding hamiltonian, and the class `PauliExpectation` to represent the entire circuit for QAOA along with the measurement operator in order to compute $\langle \psi | H_c | \psi\rangle$.

In [ ]:
def evaluate_hamiltonian(beta_gamma, hamiltonian):
    n = len(beta_gamma)//2
    circuit = create_circuit(beta_gamma[:n], beta_gamma[n:])
    meas = OperatorStateFn(hamiltonian, is_measurement=True)
    circuit_meas = PauliTrotterEvolution(trotter_mode=Suzuki(order=3, reps=1)).convert(meas @ circuit)
    circuit_expectation = PauliExpectation().convert(circuit_meas)
    
    sampler = CircuitSampler(backend=Aer.get_backend('statevector_simulator'))
    sampled_trotter_exp_op = sampler.convert(circuit_expectation)
    return sampled_trotter_exp_op.eval().real

Finally, we optimize the angles to minimize the expectation value of the cost hamiltonian `Hc`:

In [ ]:
result = minimize(evaluate_hamiltonian, np.concatenate([beta, gamma]), (Hc), method='L-BFGS-B')
result

# Analysis of the results

We create a quantum circuit using the optimal parameters found.

In [ ]:
circuit = create_circuit(result['x'][:p], result['x'][p:])
circuit = circuit.to_circuit_op().primitive

We use the `statevector_simulator` backend in order to display the state created by the circuit.

In [ ]:
backend = Aer.get_backend('statevector_simulator')
job = execute(circuit, backend)
state = np.asarray(job.result().get_statevector(circuit))
print(np.absolute(state))
print(np.angle(state))

We see that the state is approximately $\frac{1}{\sqrt{2}} \left( |00 \rangle + |11 \rangle \right)$. It corresponds to a uniform superposition of the two solutions of the classicial problem: $(\sigma_1=1$, $\sigma_2=1)$ and $(\sigma_1=-1$, $\sigma_2=-1)$

Let's now try to evaluate the operators $\sigma^Z_1$ and $\sigma^Z_2$ independently:

In [ ]:
Z0 = pauli_z(0, 1)
Z1 = pauli_z(1, 1)

In [ ]:
print(evaluate_hamiltonian(np.concatenate([beta, gamma]), Z0))
print(evaluate_hamiltonian(np.concatenate([beta, gamma]), Z1))

We see that both are approximatively equal to zero. It's expected given the state we found above and corresponds to a typical quantum behavior where $\mathbb{E}[\sigma^Z_1 \sigma^Z_2] \neq \mathbb{E}[\sigma^Z_1] \mathbb{E}[\sigma^Z_2]$